In [1]:
import pymongo

In [3]:
# Conexión al servidor local de mongodb
conex = pymongo.MongoClient()
conex.list_database_names()

['BDClientes',
 'BDPacientes',
 'BDSuCasa',
 'Provincias',
 'admin',
 'clasesabadoBD',
 'config',
 'local',
 'practica',
 'prueba']

In [4]:
# Conexión a la base de datos e inserción en la colección de documentos
db = conex.practica

In [5]:
db.list_collection_names()

['autores', 'documentos']

In [6]:
db.get_collection('autores').estimated_document_count()

2499367

In [7]:
db.get_collection('documentos').estimated_document_count()

4868818

### C1

Listado de todas las publicaciones de un autor determinado

In [19]:
author = "A Ozuna"

In [31]:
%%time
c1 = db.get_collection('autores').find({"_id":author},{"publications":1})

CPU times: user 68 µs, sys: 9 µs, total: 77 µs
Wall time: 79.9 µs


In [32]:
print("El autor {} tiene las siguientes publicaciones: {}".format(author, list(c1[0]['publications'])))

El autor A Ozuna tiene las siguientes publicaciones: ['journals/bioinformatics/OzunaLJAN20']


### C2

Número de publicaciones de un autor determinado

In [42]:
author = "(Max) Zong-Ming Cheng"

In [43]:
pipeline_c2 = [{"$match":{"_id":author}},
               {"$project": {"number_publications":{"$size":"$publications"}}}]

In [44]:
%%time
c2 = db.autores.aggregate(pipeline_c2)

CPU times: user 0 ns, sys: 1.75 ms, total: 1.75 ms
Wall time: 1.15 ms


In [45]:
print("El autor {} tiene {} publicaciones".format(author, list(c2)[0]['number_publications']))

El autor (Max) Zong-Ming Cheng tiene 2 publicaciones


### C3

Número de artículos en revista para el año 2018

In [50]:
pipeline_c3 = [
    {"$match":{"$and":[{"year":"2018"}, {"type":"article"}]}},
    {"$count":"number_articles_2018"}
]

In [51]:
%%time
c3 = db.documentos.aggregate(pipeline_c3)

CPU times: user 2.97 ms, sys: 0 ns, total: 2.97 ms
Wall time: 1.87 s


In [53]:
print("Hay {} artículos en el año 2018".format(list(c3)[0]['number_articles_2018']))

Hay 179805 artículos en el año 2018


### C4

Número de autores ocasionales, es decir, que tengan menos de 5 publicaciones en total.

In [55]:
pipeline_c4 = [
    {"$project": {"number_publications":{"$size":"$publications"}}},
    {"$match": {"number_publications":{"$gte":5}}},
    {"$count": "number_authors_with_5_or_more_publications"}
]

In [56]:
%%time
c4 = db.autores.aggregate(pipeline_c4)

CPU times: user 4.39 ms, sys: 610 µs, total: 5 ms
Wall time: 3.61 s


In [57]:
print("Hay {} autores con al menos 5 publicaciones".format(list(c4)[0]['number_authors_with_5_or_more_publications']))

Hay 565192 autores con al menos 5 publicaciones


In [18]:
print(list(c4))

[{'number_authors_with_5_or_more_publications': 565192}]


Check

In [58]:
pipeline_c4 = [
    {"$project": {"number_publications":{"$size":"$publications"}, "publications":1}},
    {"$match": {"number_publications":{"$gte":5}}},
    {"$sample": {"size": 2}}
]

In [59]:
%%time
c4 = db.autores.aggregate(pipeline_c4)

CPU times: user 66 µs, sys: 3.61 ms, total: 3.68 ms
Wall time: 3.76 s


In [60]:
print(list(c4))

[{'_id': 'Li-Yeng Sung', 'publications': ['journals/jnma/BarkerBS11', 'journals/siamnum/BrennerLS08', 'journals/siamnum/BrennerGS18', 'journals/siamnum/BrennerGGS12', 'journals/siamnum/BrennerSZZ12', 'journals/siamnum/BrennerGSZ17', 'journals/siamnum/BrennerS06', 'journals/nla/BrennerCS09', 'journals/nla/BrennerPS13', 'journals/cmam/AntoniettiDBS12', 'journals/cmam/BrennerGS17', 'journals/cmam/BrennerCS19', 'journals/cmam/BrennerCCS18', 'journals/cma/BrennerLS17', 'journals/nm/BrennerNRS17', 'journals/nm/BrennerS99', 'journals/nm/BrennerOS18', 'journals/nm/BrennerLS14', 'journals/nm/BrennerCLS08', 'journals/nm/BrennerCGS11', 'journals/corr/abs-2001-10933', 'journals/corr/abs-2003-08504', 'journals/appml/BrennerS09', 'journals/jscic/BrennerDS18', 'journals/jscic/BrennerLS09', 'journals/jscic/BarkerBPS11', 'journals/jscic/BrennerS05', 'journals/jscic/BrennerSS17', 'journals/jscic/BrennerGS09', 'journals/jscic/BrennerGS13', 'journals/jscic/BrennerGOS10', 'journals/jscic/BrennerGS16', 'jou

### C5

Número de artículos de revista (article) y número de artículos en congresos
(inproceedings) de los diez autores con más publicaciones totales.

In [191]:
pipeline_c5 = [
    {"$match": {"_id": {"$ne":None}}},
    {"$project": {"number_publications":{"$size":"$publications"}, "publications":1}},
    {"$sort": {"number_publications":-1}},
    {"$limit": 10},
    {"$unwind": "$publications"},
    {"$lookup": {"from": "documentos",
                "localField": "publications",
                "foreignField": "_id",
                "as": "publication_info"}},
    {"$match": {"$or":[{"publication_info.type": "article"},
                       {"publication_info.type": "inproceedings"}]}},
    {"$project": {"publication_type": {"$arrayElemAt": ["$publication_info.type", 0]}}},
    {"$project": {"is_article": {"$cond": [{"$eq": ["$publication_type","article"]},1,0]}, 
                  "is_inpro": {"$cond": [{"$eq": ["$publication_type","inproceedings"]},1,0]}}},
    {
        "$group":
        {
            "_id":"$_id",
            "number_articles": {"$sum": "$is_article"},
            "number_inpro": {"$sum": "$is_inpro"}
        }
    }   
]

In [192]:
%%time
c5 = db.autores.aggregate(pipeline_c5, allowDiskUse=True)

CPU times: user 1.72 ms, sys: 3.72 ms, total: 5.44 ms
Wall time: 5.97 s


In [193]:
print(list(c5))

[{'_id': 'Mohamed-Slim Alouini', 'number_articles': 839, 'number_inpro': 624}, {'_id': 'Wei Zhang', 'number_articles': 641, 'number_inpro': 707}, {'_id': 'Victor C. M. Leung', 'number_articles': 639, 'number_inpro': 533}, {'_id': 'Wen Gao 0001', 'number_articles': 403, 'number_inpro': 803}, {'_id': 'Yu Zhang', 'number_articles': 557, 'number_inpro': 675}, {'_id': 'Lajos Hanzo', 'number_articles': 852, 'number_inpro': 418}, {'_id': 'Philip S. Yu', 'number_articles': 543, 'number_inpro': 825}, {'_id': 'Wei Wang', 'number_articles': 603, 'number_inpro': 719}, {'_id': 'Wei Li', 'number_articles': 592, 'number_inpro': 661}, {'_id': 'H. Vincent Poor', 'number_articles': 1277, 'number_inpro': 601}]


### C6

Número medio de autores de todas las publicaciones que tenga en su conjunto de datos

In [202]:
pipeline_c6 = [
    {"$project":
     {
         "number_authors": {"$size": "$authors"}
     }
    }, 
    {"$group":
     {
         "_id": 0,
         "avg_number_authors": {"$avg": "$number_authors"}
     }
    }
]

In [203]:
%%time
c6 = db.documentos.aggregate(pipeline_c6)

CPU times: user 4.91 ms, sys: 620 µs, total: 5.53 ms
Wall time: 5.93 s


In [204]:
print(list(c6))

[{'_id': 0, 'avg_number_authors': 3.0612990668371665}]


### C7

Listado de coautores de un autor (Se denomina coautor a cualquier persona que haya
firmado una publicación).

In [212]:
author = "(Max) Zong-Ming Cheng"

In [248]:
pipeline_c7 = [
    {"$match":{"_id":author}},
    {"$project": {"publications":1}},
    {"$unwind": "$publications"},
    {"$lookup": {"from": "documentos",
                "localField": "publications",
                "foreignField": "_id",
                "as": "publication_info"}},
    {"$project": {"publication_authors": {"$arrayElemAt": ["$publication_info.authors", 0]}}},
    {"$unwind": "$publication_authors"},
    {"$match": {"publication_authors": {"$ne":author}}},
    {
        "$group": {
            "_id":"$_id",
            "coauthors": {"$addToSet": "$publication_authors"}
        } 
    }   
]

In [249]:
%%time
c7 = db.autores.aggregate(pipeline_c7)

CPU times: user 1.47 ms, sys: 192 µs, total: 1.66 ms
Wall time: 1.22 ms


In [250]:
print(list(c7))

[{'_id': '(Max) Zong-Ming Cheng', 'coauthors': ['Douglas G. Hayes', 'Dandan Zhang', 'Xiaohan Yang', 'Mark Davis', 'Gerald A. Tuskan', 'Peng Gao', 'Joseph J. Bozell', 'Joshua S. Yuan', 'Kristen L. Reichel', 'Nathan R. Stewart', 'Robert W. Sykes', 'Zhanyou Xu', 'Jun Hu', 'Weibing Shi', 'Bin Cai', 'Ryan D. Syrenne', 'Xia Ye 0002', 'Crissa Doeppke', 'Xin Zhou', 'C. Neal Stewart Jr.', 'Nicole Labbe', 'Jason N. Burris']}]


### C8

Edad de los 5 autores con un periodo de publicaciones más largo (Se considera la Edad
de un autor al número de años transcurridos desde la fecha de su primera publicación
hasta la última registrada).

In [316]:
db.documentos.create_index('year')

'year_1'

In [317]:
list(db.documentos.list_indexes())

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'practica.documentos')]),
 SON([('v', 2), ('key', SON([('year', 1)])), ('name', 'year_1'), ('ns', 'practica.documentos')])]

In [299]:
list(db.autores.list_indexes())

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'practica.autores')])]

In [300]:
pipeline_c8 = [
    {"$match": {"_id": {"$ne":None}}},
    {"$project": {"publications":1}},
    {"$unwind": "$publications"},
    {"$lookup": {"from": "documentos",
                "localField": "publications",
                "foreignField": "_id",
                "as": "publication_info"}},
    {"$project": {"publication_year": {"$arrayElemAt": ["$publication_info.year", 0]}}},
    {
        "$group": {
            "_id": "$_id",
            "first_publication_year": {"$min": "$publication_year"},
            "last_publication_year": {"$max": "$publication_year"}
        }
    },
    {"$project": 
     {
        "age": {"$subtract": ["$last_publication_year", "$first_publication_year"]}
     }
    },
    {"$limit":5}
]

In [318]:
# Cambiamos min max por sort y first, last (el otro no tira)
pipeline_c8 = [
    {"$match": {"_id": {"$ne":None}}},
    {"$project": {"publications":1}},
    {"$unwind": "$publications"},
    {"$lookup": {"from": "documentos",
                "localField": "publications",
                "foreignField": "_id",
                "as": "publication_info"}},
    {"$project": {"publication_year": {"$arrayElemAt": ["$publication_info.year", 0]}}},
    {"$sort": {"publication_year":1}},
    {
        "$group": {
            "_id": "$_id",
            "first_publication_year": {"$first": "$publication_year"},
            "last_publication_year": {"$last": "$publication_year"}
        }
    },
    {"$project": 
     {
        "age": {"$subtract": ["$last_publication_year", "$first_publication_year"]}
     }
    },
    {"$sort": {"age":-1}},
    {"$limit":5}
]

In [319]:
%%time
db.autores.aggregate(pipeline_c8, allowDiskUse=True)

KeyboardInterrupt: 

In [290]:
print(list(c8)[:10])

[{'_id': '"Johann" Sebastian Rudolph', 'publication_year': '2019'}, {'_id': "'Anau Mesui", 'publication_year': '2019'}, {'_id': "'Maseka Lesaoana", 'publication_year': '2001'}, {'_id': "'Maseka Lesaoana", 'publication_year': '2015'}, {'_id': "'Niran Adetoro", 'publication_year': '2019'}]


### C9

Número de autores novatos, es decir, que tengan una Edad menor de 5 años. Se
considera la Edad de un autor al número de años transcurridos desde la fecha de su
primera publicación hasta la última registrada.

Y si creamos primero una colección que sea la edad de cada autor?

In [ ]:
pipeline_c9 = [
    {"$match": {"_id": {"$ne":None}}},
    {"$project": {"publications":1}},
    {"$unwind": "$publications"},
    {"$lookup": {"from": "documentos",
                "localField": "publications",
                "foreignField": "_id",
                "as": "publication_info"}},
    {"$project": {"publication_year": {"$arrayElemAt": ["$publication_info.year", 0]}}},
    {
        "$group": {
            "_id": "$_id",
            "first_publication_year": {"$min": "$publication_year"},
            "last_publication_year": {"$max": "$publication_year"}
        }
    },
    {"$project": 
     {
        "age": {"$subtract": ["$last_publication_year", "$first_publication_year"]}
     }
    },
    {"$match": {"age": {"$lt": 5}}},
    {"$count": "number_authors_with_less_than_5_years"}
]

In [ ]:
%%time
c9 = db.autores.aggregate(pipeline_c9, allowDiskUse=True)

In [ ]:
print(list(c9))

### C10

Porcentaje de publicaciones en revistas con respecto al total de publicaciones.

In [307]:
pipeline_c10 = [
    {"$project": {"is_article": {"$cond": [{"$eq": ["$type", "article"]},1,0]}}},
    {
        "$group": 
        {
            "_id": 0,
            "number_articles": {"$sum": "$is_article"},
            "total_number_publications": {"$sum": 1}
        }
    },
    {"$project":{"perc_articles": {"$divide": ["$number_articles", "$total_number_publications"]}}},
    {"$project": {"perc_articles_100": {"$multiply": ["$perc_articles", 100]}}}
    
]

In [308]:
%%time
c10 = db.documentos.aggregate(pipeline_c10)

CPU times: user 2.55 ms, sys: 0 ns, total: 2.55 ms
Wall time: 4.54 s


In [309]:
print(list(c10))

[{'_id': 0, 'perc_articles_100': 45.89996997217805}]
